# Random Forest analysis

Why do we use to use a Random Forest analysis on our volatiles & acylsugar datasets?

Random Forests have unique features:
- use a random set of variables to make a split at each node for each decision tree.
- this has potential ...


## Steps to perform for a canonical RF analysis:

- **Cross-validate**: to estimate the robustness of the RF model by splitting multiple times your dataset into a train and a test set.
- **Tuning the hyper-parameters**: split the train set into a validation set (fine tune the HF) and a train set.


## Scenario for our dataset

Use all the data at our disposal and perform k-fold cross validation. Then average the feature importances for each model to get the most important features out. 

Then, perform the same analysis but with permuted variables to get a random distribution for each feature importance. That would also to compare each feature importance with the permuted ones and compute a p-value.


# Data import and transformation
The script imports a table containing:
- the phenotypic classes for whitefly and thrips of the 20 tomato genotypes.
- the stem trichome volatiles normalised values.

In [1]:
import pandas as pd
df = pd.read_csv("../Figure7_PLS-DA/pheno_terpenoids.tsv",sep="\t",index_col=0)
df.head()

,wf,thrips,5.541_67.0557,7.060_95.0168,9.272_93.0728,9.472_91.0566,9.653_91.0565,10.148_93.0730,10.581_105.0363,10.873_119.0877,...,26.529_131.0872,26.595_91.0568,26.689_109.1032,26.803_111.0836,26.833_81.0727,27.083_97.0317,27.996_91.0565,28.382_97.0323,30.160_91.0569,32.503_159.8606
sample,,,,,,,,,,,,,,,,,,,,,
S lyc MM,non-toxic,non-toxic,0.00000,0.0,0.000,0.0000,88210.77165,0.00000,0.00000,32617.13711,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
S per LA1278,non-toxic,toxic,234383.78650,0.0,0.000,0.0000,131824.19000,0.00000,800290.02030,0.00000,...,0.0,0.0,0.0,9153.957334,0.0,0.0,0.0,0.0,0.0,0.0
S hua LA1364,non-toxic,non-toxic,0.00000,0.0,104528.752,45335.4665,27304.44693,90430.09761,0.00000,0.00000,...,0.0,0.0,0.0,5561.180230,0.0,0.0,0.0,0.0,0.0,0.0
S che LA1401,non-toxic,toxic,0.00000,0.0,0.000,0.0000,457891.15640,0.00000,32459.72293,0.00000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
S pim LA1578,non-toxic,toxic,12809.66965,0.0,0.000,0.0000,221391.28520,0.00000,0.00000,24737.81096,...,0.0,0.0,0.0,9495.496480,0.0,0.0,0.0,0.0,0.0,0.0


# Whitefly analysis

## Train and test splits

In [19]:
# get whitefly classes
y = df["wf"].tolist()


stratify_info = df['wf'].map({'non-toxic': 0, 'toxic': 1})

In [20]:
stratify_info

sample
S lyc MM          0
S per LA1278      0
S hua LA1364      0
S che LA1401      0
S pim LA1578      0
S hab LA1718      1
S hab LA1777      1
S chm LA1840      0
S per LA1954      1
S neo LA2133      0
S arc LA2172      0
S chm LA2695      1
S lyc LA4024      0
S hab LA0407      0
S pen LA0716      1
S neo LA0735      0
S hab PI134418    1
S hab LYC4        1
S hab PI127826    1
Name: wf, dtype: int64

In [3]:
# get matrix X of volatile values
X = df.iloc[:,2:]
X.head()

,5.541_67.0557,7.060_95.0168,9.272_93.0728,9.472_91.0566,9.653_91.0565,10.148_93.0730,10.581_105.0363,10.873_119.0877,11.056_91.0567,11.572_91.0564,...,26.529_131.0872,26.595_91.0568,26.689_109.1032,26.803_111.0836,26.833_81.0727,27.083_97.0317,27.996_91.0565,28.382_97.0323,30.160_91.0569,32.503_159.8606
sample,,,,,,,,,,,,,,,,,,,,,
S lyc MM,0.00000,0.0,0.000,0.0000,88210.77165,0.00000,0.00000,32617.13711,0.00000,0.000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
S per LA1278,234383.78650,0.0,0.000,0.0000,131824.19000,0.00000,800290.02030,0.00000,55971.00761,0.000,...,0.0,0.0,0.0,9153.957334,0.0,0.0,0.0,0.0,0.0,0.0
S hua LA1364,0.00000,0.0,104528.752,45335.4665,27304.44693,90430.09761,0.00000,0.00000,0.00000,624085.817,...,0.0,0.0,0.0,5561.180230,0.0,0.0,0.0,0.0,0.0,0.0
S che LA1401,0.00000,0.0,0.000,0.0000,457891.15640,0.00000,32459.72293,0.00000,34284.95912,0.000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
S pim LA1578,12809.66965,0.0,0.000,0.0000,221391.28520,0.00000,0.00000,24737.81096,0.00000,0.000,...,0.0,0.0,0.0,9495.496480,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# Let's split our data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=40,stratify=stratify_info)

## Initiate a Random Forest model

In [23]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000,class_weight={"toxic":1/8,"non-toxic":1/11})
rf

RandomForestClassifier(bootstrap=True,
            class_weight={'toxic': 0.125, 'non-toxic': 0.09090909090909091},
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=None, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

## 6-fold cross validation (n=19 samples)

Will this change every time you run it?

In [24]:
from sklearn.model_selection import cross_val_score

# performs 6 fold cross-validation 
cv_scores = cross_val_score(rf,X,y,cv=6)
print("Accuracy: %0.2f (+/- %0.2f)" % (cv_scores.mean(), cv_scores.std() * 2))

Accuracy: 0.78 (+/- 0.46)


## Fit model and compute metrics

In [25]:
# fit a model
rf = rf.fit(X_train,y_train)

# predict class values
y_pred = rf.predict(X_test)
y_pred = list(y_pred)
y_pred

['non-toxic', 'non-toxic', 'non-toxic', 'non-toxic', 'non-toxic', 'non-toxic']

In [29]:
y_pred

['non-toxic', 'non-toxic', 'non-toxic', 'non-toxic', 'non-toxic', 'non-toxic']

In [26]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred)) 

              precision    recall  f1-score   support

   non-toxic       0.50      1.00      0.67         3
       toxic       0.00      0.00      0.00         3

   micro avg       0.50      0.50      0.50         6
   macro avg       0.25      0.50      0.33         6
weighted avg       0.25      0.50      0.33         6



/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
# get the target labels (it is an output from the classifier)
target_names=rf.classes_

def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
print('Confusion matrix, without normalization')
print(cm)
plt.figure()
plot_confusion_matrix(cm)

cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Normalized confusion matrix')
print(cm_normalized)
plt.figure()
plot_confusion_matrix(cm_normalized, title='Normalized confusion matrix')

plt.show()

NameError: name 'plt' is not defined